## Az eredményeket mentsd a src/weekly modul-ba weekly_test_6.py néven

#### Használható modulok: pathlib, pandas, typing, str, statsmodels

In [ ]:
from pathlib import Path

In [ ]:
datalib = Path.cwd().parent.joinpath('data')

In [ ]:
"""
1., Olvasd be a data mappa sp500.parquet nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen fastparquet
"""

In [2]:
import pandas as pd

file_path = "../data/sp500.parquet"

df = pd.read_parquet(file_path, engine="fastparquet")


In [ ]:
"""
2., Olvasd be az ff_factors.parquet fájlt egy DataFrame-be. A betöltéshez használt engine paramétere legyen fastparquet
"""

In [4]:
import pandas as pd

file_path = "../data/ff_factors.parquet"

df = pd.read_parquet(file_path, engine="fastparquet")


In [ ]:
"""
3., Kapcsold össze a két DataFrame-t egy új DataFrame-be. Az összekapcsolás módja, hogy a hozam adatokra balról kapcsoljuk rá a factor adatokat a 'Date' elsődleges kulcs alapján.
"""

![Joined data](../resources/weekly6/joined_data.jpg)

In [6]:
# Első DataFrame (sp500.parquet) beolvasása
sp500_df = pd.read_parquet("../data/sp500.parquet", engine="fastparquet")

# Második DataFrame (ff_factors.parquet) beolvasása
ff_df = pd.read_parquet("../data/ff_factors.parquet", engine="fastparquet")

# Összekapcsolás a 'Date' kulcs alapján
merged_df = sp500_df.merge(ff_df, on='Date', how='left')


In [ ]:
"""
4., Készíts egy új 'Excess Return' nevű oszlopot, ami a havi hozamok és a kockázat mentes hozam (RF) különbsége
"""

In [9]:
# Módosított DataFrame létrehozása az "Excess Return" oszloppal
merged_df['Excess Return'] = merged_df['Monthly Returns'] - merged_df['RF']


In [ ]:
"""
5., Rendezd sorba dátum szerint az adatokat, majd generálj egy új oszlopot ('ex_ret_1'), amely minden ticker ('Symbol') esetén 1-el eltolja az Excess Return értékeit olyan módon, hogy minden sorban szerepeljen a következő időszaki Excess Return érték. 
"""

![new column](../resources/weekly6/ex_ret_1.jpg)

In [10]:
# Dátum szerinti rendezés,  ezt raktam be
merged_df = merged_df.sort_values(by=['Symbol', 'Date'])

# Az "Excess Return" értékek 1-el való eltolása minden "Symbol" csoporton belül
merged_df['ex_ret_1'] = merged_df.groupby('Symbol')['Excess Return'].shift(-1)


In [11]:
# Első lépés: rendezzük sorba a DataFrame-t dátum szerint
merged_df.sort_values(by=['Symbol', 'Date'], inplace=True)

# Második lépés: csoportosítsuk a DataFrame-t a 'Symbol' oszlop alapján
grouped = merged_df.groupby('Symbol')

# Harmadik lépés: alkossunk egy új oszlopot, amely tartalmazza az eltolást
merged_df['ex_ret_1'] = grouped['Excess Return'].shift(-1)


In [ ]:
"""
6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer törlöd az össze olyan sort, amely az 'ex_ret_1' oszlopban hiányos, majd ezt követően, törlöd az összes olyan sort, ami a 'HML' oszlopban hiányos
"""

In [12]:
# Hiányzó értékek törlése az 'ex_ret_1' oszlopban
merged_df = merged_df.dropna(subset=['ex_ret_1'])

# Hiányzó értékek törlése a 'HML' oszlopban
merged_df = merged_df.dropna(subset=['HML'])

In [ ]:
# Töröljük a 'Symbol' oszlopot
merged_df.drop('Symbol', axis=1, inplace=True)

# Válasszuk ki az 'AMZN' részvényhez tartozó sorokat
amazon_df = merged_df[merged_df['Symbol'] == 'AMZN']

In [ ]:
"""
A meglévő adatokból válaszd ki a Amazon részvényhez tartozó sorokat (AMZN) és töröld a tickereket tartalmazó oszlopot

7., Készíts egy új LinearRegressionSM elnevezésű osztályt. Definiáld benne a __init__ nevű függvényt, amely bemenetként 2 DataDrame-t kap amelyeket ments le a left_hand_side és right_hand_side elnevezésű változókba. Az egyik DataFrame fogja tartalmazni a következő hónap többlet hozamait (left_hand_side), a másik a piaci hozamokat (Mkt-RF), az SMB és a HML értékeket (right_hand_side).

8., Egésztsd ki az osztályt egy fit metódussal, ami meghívja a statsmodels segítségével OLS alapon becsült modellt illeszt az osztály adattagjaira. A becslésből visszakapott modellt mentse le az osztály _model adattagjába. Figyelj oda, hogy a regresszió futtatása során konstans (alfa / béta_0) is szerepeljen a predictor változók között.

9., Egészítsd ki az osztályt egy get_params metódussal, ami visszaadja a becsült modellt béta paramétereinek értékeit. A visszakapott pandas Series típusú adatban az oszlop neve legyen 'Beta coefficients'

10., Egészítsd ki az osztályt egy get_pvalues metódussal, ami visszaadja a becsült modell paraméterekhez tartozó p értékeket. A visszakapott pandas Series típusú adatban az oszlop neve legyen: P-values for the corresponding coefficients

11., Egészítsd ki az osztályt egy get_wald_test_result metódussal, ami visszaadja a bemeneti restrikciós mátrix alapján számolt F és p értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: F-value: fvalue, p-value: pvalue, ahol az fvalue és pvalue helyére 3 tizedesjegyre kerekítve adja meg a hozzájuk tartozó értékeket.

12., Egészítse ki az osztályt egy get_model_goodness_values metódussal, ami visszadja a módosított R-négyzet, az Akaike és a Bayes információs kritériumok értékét. A visszatérési típus string legyen, a visszaadandó szöveg: Adjusted R-squared: ars, Akaike IC: ak, Bayes IC: by, ahol ars, ak és by helyére 3 tizedesjegyre kerekítve adja meg a hozzájuk tartozó értékeket.

A létrehozott osztályt és a hozzátartozó metódusokat mentsd le a src.linear_regression mappába, a LinearRegressions.py fájlba.
"""

In [ ]:
# Töröljük a 'Symbol' oszlopot
merged_df.drop('Symbol', axis=1, inplace=True)

# Válasszuk ki az 'AMZN' részvényhez tartozó sorokat
amazon_df = merged_df[merged_df['Symbol'] == 'AMZN']


In [ ]:
# Amazon részvényhez tartozó sorok kiválasztása (AMZN)
amazon_df = merged_df[merged_df['Symbol'] == 'AMZN']

# Ticker oszlop törlése
amazon_df = amazon_df.drop(columns=['Symbol'])


In [ ]:
class LinearRegressionSM:
    def __init__(self, left_hand_side, right_hand_side):
        self.left_hand_side = left_hand_side
        self.right_hand_side = right_hand_side
        self._model = None

    def fit(self):
        formula = "left_hand_side ~ Mkt_RF + SMB + HML"
        model = ols(formula=formula, data=self.right_hand_side).fit()
        self._model = model

    def get_params(self):
        # Az illesztett modell koefficienseinek lekérése és elnevezése
        beta_params = self._model.params.rename("Beta coefficients")
        return beta_params


    def get_pvalues(self):
        p_values = self._model.pvalues.rename("P-values for the corresponding coefficients")
        return p_values


    def get_wald_test_result(self, constraint_matrix):
        wald_result = self._model.wald_test(constraint_matrix)
        f_value = wald_result.statistic[0, 0]
        p_value = wald_result.pvalue

        result_text = f"F-value: {f_value:.3f}, p-value: {p_value:.3f}"

        return result_text